In [1]:
import pandas as pd
import requests
from io import StringIO

# Criação do dataFrame dos alunos
# ID do arquivo no Google Drive
file_id = '1inLJ0QY0vOhD_2CKujdLEUvF4MnPbAi5'

# URL modificada para forçar o download do arquivo
url = f"https://drive.google.com/uc?id={file_id}"

# Tentando obter o arquivo com requests
try:
    response = requests.get(url)
    response.raise_for_status()  # Lança um erro para respostas não-sucedidas
    # Usando StringIO para converter o texto em um arquivo em memória e, então, lendo com o Pandas
    csv_raw = StringIO(response.text)
    ecom = pd.read_csv(csv_raw)
except requests.RequestException as e:
    print(f"Erro ao acessar o arquivo: {e}")

In [2]:
ecom

,ID,corredor_armazem,modo_envio,numero_chamadas_cliente,avaliacao_cliente,custo_produto,compras_anteriores,prioridade_produto,genero,desconto,peso_gramas,entregue_no_prazo
0,1,D,Aviao,4,2,177,3,baixa,F,44,1233,1
1,2,F,Aviao,4,5,216,2,baixa,M,59,3088,1
2,3,A,Aviao,2,2,183,4,baixa,M,48,3374,1
3,4,B,Aviao,3,3,176,4,media,M,10,1177,1
4,5,C,Aviao,2,2,184,3,media,F,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10995,10996,B,Navio,4,1,232,5,media,F,6,1247,0
10996,10997,C,Navio,5,4,242,5,baixa,F,4,1155,0
10997,10998,F,Navio,5,2,223,6,media,M,2,1210,0
10998,10999,D,Navio,2,5,155,5,baixa,F,6,1639,0


Verificar as informações do conjunto de dados

In [3]:
ecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       11000 non-null  int64 
 1   corredor_armazem         11000 non-null  object
 2   modo_envio               11000 non-null  object
 3   numero_chamadas_cliente  11000 non-null  int64 
 4   avaliacao_cliente        11000 non-null  int64 
 5   custo_produto            11000 non-null  int64 
 6   compras_anteriores       11000 non-null  int64 
 7   prioridade_produto       11000 non-null  object
 8   genero                   11000 non-null  object
 9   desconto                 11000 non-null  int64 
 10  peso_gramas              11000 non-null  int64 
 11  entregue_no_prazo        11000 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB


## Análise Exploratória

### Separação de Variáveis numéricas das variáveis categóricas

In [4]:
nums = ['numero_chamadas_cliente', 'custo_produto', 'compras_anteriores', 'desconto', 'peso_gramas']

In [5]:
categ = ['corredor_armazem', 'modo_envio', 'avaliacao_cliente', 'prioridade_produto', 'genero']

In [6]:
target = ['entregue_no_prazo']

Para as variáveis numéricas podemos usar o método describe para ver um resumo dos dados.

In [7]:
ecom[nums].describe()

,numero_chamadas_cliente,custo_produto,compras_anteriores,desconto,peso_gramas
count,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000
mean,4.054455,210.200909,3.567727,13.372545,3633.844455
std,1.141438,48.062985,1.522852,16.204943,1635.402722
min,2.000000,96.000000,2.000000,1.000000,1001.000000
25%,3.000000,169.000000,3.000000,4.000000,1839.000000
50%,4.000000,214.000000,3.000000,7.000000,4149.000000
75%,5.000000,251.000000,4.000000,10.000000,5050.000000
max,7.000000,310.000000,10.000000,65.000000,7846.000000


Para as variáveis categóricas isso não faz muito sentido, então devemos procurar outras formas de descrever o conjunto de dados.

In [8]:
for coluna in categ:
    display(ecom[coluna].value_counts())

corredor_armazem
F    3666
D    1834
C    1834
A    1833
B    1833
Name: count, dtype: int64

modo_envio
Navio       7463
Aviao       1777
Caminhao    1760
Name: count, dtype: int64

avaliacao_cliente
3    2239
1    2235
4    2189
5    2171
2    2166
Name: count, dtype: int64

prioridade_produto
baixa    5298
media    4754
alta      948
Name: count, dtype: int64

genero
F    5546
M    5454
Name: count, dtype: int64

Obtendo informações do nosso target

In [9]:
ecom[target].value_counts()

entregue_no_prazo
1                    6563
0                    4437
Name: count, dtype: int64

## Limpeza de Dados

Vamos começar checando os Null e NaN.

In [10]:
ecom.isnull().sum()

ID                         0
corredor_armazem           0
modo_envio                 0
numero_chamadas_cliente    0
avaliacao_cliente          0
custo_produto              0
compras_anteriores         0
prioridade_produto         0
genero                     0
desconto                   0
peso_gramas                0
entregue_no_prazo          0
dtype: int64

In [11]:
ecom.isna().sum()

ID                         0
corredor_armazem           0
modo_envio                 0
numero_chamadas_cliente    0
avaliacao_cliente          0
custo_produto              0
compras_anteriores         0
prioridade_produto         0
genero                     0
desconto                   0
peso_gramas                0
entregue_no_prazo          0
dtype: int64

Não existem valores ausentes e 'Not a Number' no conjunto de dados, mas será que nossos problemas estão resolvidos ? 

Também podemos checar registros duplicados.

In [12]:
ecom.duplicated().sum()

0

### Estudo de Outliers

Outliers são pontos de dados que se distanciam significativamente dos demais em um conjunto, representando valores incomuns. Esses valores atípicos podem afetar análises estatísticas, possivelmente ocultando descobertas importantes ou distorcendo resultados. A identificação de outliers não segue regras estatísticas rígidas, dependendo mais do conhecimento específico da área e da análise do processo de coleta de dados. Embora não haja uma definição matemática precisa, métodos e testes estatísticos ajudam a detectar esses pontos discrepantes, que são cruciais por seu potencial de impactar análises estatísticas adversamente.

Vamos começar a análise olhando outra vez para as variáveis numéricas

In [13]:
ecom[nums].describe()

,numero_chamadas_cliente,custo_produto,compras_anteriores,desconto,peso_gramas
count,11000.000000,11000.000000,11000.000000,11000.000000,11000.000000
mean,4.054455,210.200909,3.567727,13.372545,3633.844455
std,1.141438,48.062985,1.522852,16.204943,1635.402722
min,2.000000,96.000000,2.000000,1.000000,1001.000000
25%,3.000000,169.000000,3.000000,4.000000,1839.000000
50%,4.000000,214.000000,3.000000,7.000000,4149.000000
75%,5.000000,251.000000,4.000000,10.000000,5050.000000
max,7.000000,310.000000,10.000000,65.000000,7846.000000


O atributo desconto está com características interessantes:

 - a média(mean): 13.372545
 - o desvio-padrão(std): 16.204943
 - o menor desconto (min): 1.000000
 - o maior desconto (max): 65.00000

 Além disso 75% dos produtos estão com desconto menor do que 10%, ou seja, a maioria dos produtos está com desconto de até 10%, assim podemos concluir que descontos muito maiores que 10% podem sem outliers.

In [14]:
ecom.desconto.describe()

count    11000.000000
mean        13.372545
std         16.204943
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         65.000000
Name: desconto, dtype: float64

Para aprimorar a análise de outliers, uma técnica é calcular o limite em que observações podem ser consideradas atípicas usando a fórmula: 

Limite = média ± 3*desvio padrão. 

Esse método baseia-se na regra empírica, **presumindo que os dados seguem uma distribuição normal**, onde aproximadamente 99,7% dos dados encontram-se dentro de três desvios padrão da média. Observações fora desse limite são potencialmente outliers, sugerindo variações atípicas que podem necessitar de investigação adicional para determinar suas causas ou validade.

In [15]:
limiteSuperior = ecom.desconto.mean() + 3*ecom.desconto.std()
limiteInferior = ecom.desconto.mean() - 3*ecom.desconto.std()

Determinando os Outliers:

In [16]:
outliers = ecom[(ecom.desconto <= limiteInferior) | (ecom.desconto >= limiteSuperior)]
outliers.head()

,ID,corredor_armazem,modo_envio,numero_chamadas_cliente,avaliacao_cliente,custo_produto,compras_anteriores,prioridade_produto,genero,desconto,peso_gramas,entregue_no_prazo
36,37,D,Navio,3,5,137,4,alta,F,62,1477,1
60,61,D,Navio,3,1,221,3,baixa,F,64,2794,1
62,63,A,Navio,5,1,105,6,media,M,62,1292,1
111,112,B,Caminhao,4,2,239,3,baixa,F,65,3679,1
122,123,A,Caminhao,4,2,160,3,baixa,M,64,1714,1


In [17]:
print("O tamanho do dataset original: ", len(ecom))
print("Os outliers do atributo desconto: ", len(outliers))

O tamanho do dataset original:  11000
Os outliers do atributo desconto:  181


Podemos determinar o conjunto dos valores sem outliers com a negação da operação acima:

In [18]:
dfNormDesc = ecom[(ecom.desconto > limiteInferior) & (ecom.desconto < limiteSuperior)]
dfNormDesc.head()

,ID,corredor_armazem,modo_envio,numero_chamadas_cliente,avaliacao_cliente,custo_produto,compras_anteriores,prioridade_produto,genero,desconto,peso_gramas,entregue_no_prazo
0,1,D,Aviao,4,2,177,3,baixa,F,44,1233,1
1,2,F,Aviao,4,5,216,2,baixa,M,59,3088,1
2,3,A,Aviao,2,2,183,4,baixa,M,48,3374,1
3,4,B,Aviao,3,3,176,4,media,M,10,1177,1
4,5,C,Aviao,2,2,184,3,media,F,46,2484,1


In [19]:
print("O tamanho do dataset sem outliers no atributo desconto: ", len(dfNormDesc))

O tamanho do dataset sem outliers no atributo desconto:  10819


Agora podemos olhar para o describe da variável desconto

In [20]:
dfNormDesc.desconto.describe()

count    10819.000000
mean        12.536186
std         14.981972
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         61.000000
Name: desconto, dtype: float64

**Exercício:** Usando outras ferramentas para resolver o mesmo problema:

Vamos usar a biblioteca SciPy para resolver esse problema

In [29]:
from scipy.stats import zscore

zscore_df = ecom[nums].apply(zscore)

semOutliers_df = ecom[(zscore_df.abs()<3).all(axis=1)]

In [30]:
semOutliers_df.desconto.describe()

count    10643.000000
mean        12.528047
std         14.991968
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         61.000000
Name: desconto, dtype: float64